# FX Carry Trade Project
Fx carry is a tried and tested staple of FX strategies. It involves borrowing in a low interest rate currency and investing in a high interest rate currency. You profit from what is called the "interest rate differential". So all we need to do is find a pair of currencies with historically different rates. This project will simulate this strategy over time. I obtain/approximate short-term interest rates for the the two currencies and the historical FX rates. Then, I calculate the daily/monthly returns which is simply the interest earned from the diffrential plus any gain/loss from currency movements. I then compare this strategy to a simple buy-and-hold of the currencies to illustrate its risk/return profile.

This project itself is rather simple, but I mainly chose this project to "quantify" my experience at LSE trading society's Emerging Markets subcommittee. Although the currencies here are not from emerging economies, most of my strategies in the subcommittee involved FX trades, which is what I build on here.

I downloaded fx csv data from FRED (5Y time frame, daily data)
I downloaded rates csv data from BIS (5Y time frame, monthly data)

#### First I start off the basic imports:

In [1]:
import os
import os, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### Parameters and Paths:

In [2]:
top_n = 3             # long top-n by carry
bottom_n = 3             #short bottom-n by carry
trans_cost = 1.0           #transaction cost per unit turnover (measured in bps)

target_vol = 0.10         # annualized target portfolio vol
vol_window = 20            # rolling window (days) for realized vol
lev_min = 0.2           # leverage floor
lev_max = 2.0           # leverage cap

fx_path = "data/fx.csv"    #wide daily levels for various currencies
rates_path = "data/rates.csv"  #monthly policy rates for various currencies
out_dir = "output"
os.makedirs("data", exist_ok=True)
os.makedirs(out_dir, exist_ok=True)

#### Build DataFrames from the FRED and BIS CSVs:
##### Firstly for FRED:
Because some of the FX data is swapped, we want to swap them to have them all in form USD per XXX. It simply makes it easier to code with one "master currence" that the others are quoted in terms of.
The folder layout on GitHub for reference is:
- ..data/
- ....fred/
- ......DEXUSEU.csv
- ......DEXUSUK.csv
- ......etc
- ....bis/
- ......bis_EUR.scv
- ......bis_GBP.csv
- ......etc

Our outputs will be
- `fx` which is daily FX levels (with columns `EURUSD`, `GBPUSD`, etc)
- `rates_m` which is monthly polciy rates (columns are `EUR, GBP, ...`)
- `rates_fx` which is daily policy rates aligned to `fx`'s index and columns (which just makes the dataset clearer)

In [3]:
fred_dir = "data/fred"
bis_dir = "data/bis"

#FRED FX mapping: final column name maps to a tuple (filename, invert_needed) where invert_needed=True means we need to inver from XXXYYY to YYYXXX
fred_map = {
    "EURUSD": ("DEXUSEU.csv", False),  # USD per EUR
    "GBPUSD": ("DEXUSUK.csv", False),  # USD per GBP
    "AUDUSD": ("DEXUSAL.csv", False),  # USD per AUD
    "NZDUSD": ("DEXUSNZ.csv", False),  # USD per NZD
    "JPYUSD": ("DEXJPUS.csv", True),   # JPY per USD 
    "CHFUSD": ("DEXSZUS.csv", True),   # CHF per USD 
    "CADUSD": ("DEXCAUS.csv", True),   # CAD per USD 
    "NOKUSD": ("DEXNOUS.csv", True),   # NOK per USD 
    "SEKUSD": ("DEXSDUS.csv", True),   # SEK per USD
}

In [4]:
def read_fred(path, out_col, invert):
    if not os.path.exists(path):
        print(f"[FRED] missing {os.path.basename(path)} -> skip {out_col}")
        return None
    df = pd.read_csv(path, na_values=["."], parse_dates=["DATE"])
    # FRED csv format is usually: DATE + one data column
    series_col = [c for c in df.columns if c != "DATE"][0]
    df = df.rename(columns={"DATE": "date", series_col: out_col})
    df[out_col] = pd.to_numeric(df[out_col], errors="coerce")
    df = df.sort_values("date").set_index("date")  
    #for columns that need to be invered, we just reciprocate them
    if invert:
        df[out_col] = 1.0 / df[out_col].replace(0, np.nan)  
    return df[[out_col]] 

In [8]:
#this function will now build the dataframe for fred.
def build_fx_df():
    frames = []  #we start the list that will contain each single series dataframe, ie one per currency pair
    for col, (fname, inv) in fred_map.items():
        f = read_fred(os.path.join(fred_dir, fname), col, inv)  #builds the full file path (folder + file name) and calls our `read_fred()`
        if f is not None:
            frames.append(f)
    if not frames:
        raise FileNotFoundError("No FRED series found under data/fred/.")
    #Next we stitch all these 1-column data frames side by side into a single table, aligning them by dates        
    fx_local = pd.concat(frames, axis=1).sort_index().ffill()   #ffill() forward fills missing values, as FRED has gaps (it uses previous value for blanks)
    fx_local.index.name = "date"
    # keep columns in our order defined in `fred_map`
    fx_local = fx_local[[c for c in fred_map if c in fx_local.columns]] 
    return fx_local

In [9]:
build_fx_df()

[FRED] missing DEXUSEU.csv -> skip EURUSD
[FRED] missing DEXUSUK.csv -> skip GBPUSD
[FRED] missing DEXUSAL.csv -> skip AUDUSD
[FRED] missing DEXUSNZ.csv -> skip NZDUSD
[FRED] missing DEXJPUS.csv -> skip JPYUSD
[FRED] missing DEXSZUS.csv -> skip CHFUSD
[FRED] missing DEXCAUS.csv -> skip CADUSD
[FRED] missing DEXNOUS.csv -> skip NOKUSD
[FRED] missing DEXSDUS.csv -> skip SEKUSD


FileNotFoundError: No FRED series found under data/fred/.

##### Now for BIS:

In [1]:
# --- BIS per-country policy rates (percent or decimal depending on export)
DATE_CANDS  = ["TIME_PERIOD","TIME","DATE","Obs Period","Period","REF_DATE"]
VALUE_CANDS = ["OBS_VALUE","Value","Obs Value","OBS_VALUE (Percent per annum)"]

In [2]:
def pick_col(df, cands):
    low = {c.lower(): c for c in df.columns}
    for c in cands:
        if c.lower() in low:
            return low[c.lower()]
    raise ValueError(f"Missing one of {cands}; got {list(df.columns)}")

In [ ]:
def read_bis_file(path, code):
    df = pd.read_csv(path)
    dcol = pick_col(df, DATE_CANDS)
    vcol = pick_col(df, VALUE_CANDS)
    df = df[[dcol, vcol]].copy()
    df[dcol] = pd.to_datetime(df[dcol], errors="coerce")
    df[vcol] = pd.to_numeric(df[vcol], errors="coerce")
    df = df.dropna(subset=[dcol, vcol]).sort_values(dcol)
    df = df.drop_duplicates(subset=[dcol], keep="last")
    df = df.set_index(dcol).rename(columns={vcol: code})
    return df[[code]]

In [ ]:
def build_rates_m_df():
    files = [f for f in os.listdir(bis_dir) if re.match(r"bis_[A-Za-z]{3}\.csv$", f)]
    if not files:
        raise FileNotFoundError("No BIS files like bis_EUR.csv found in data/bis/")
    frames = []
    for fname in files:
        code = fname.split("_")[1].split(".")[0].upper()  # bis_EUR.csv -> EUR
        try:
            s = read_bis_file(os.path.join(bis_dir, fname), code)
            frames.append(s)
        except Exception as e:
            print(f"[BIS] skip {fname}: {e}")
    if not frames:
        raise RuntimeError("No valid BIS series parsed.")
    rates_local = pd.concat(frames, axis=1).sort_index()
    # if given in percent (e.g. 5.25), convert to decimals
    med = rates_local.median(numeric_only=True).median()
    if pd.notna(med) and med > 1.0:
        rates_local = rates_local / 100.0
    rates_local.index.name = "date"
    # standard order (only those present)
    wanted = ["EUR","GBP","AUD","NZD","JPY","CHF","CAD","NOK","SEK"]
    rates_local = rates_local[[c for c in wanted if c in rates_local.columns]]
    return rates_local

# --- run + align
fx = build_fx_df()          # daily FX levels, columns like EURUSD, GBPUSD, ...
rates_m = build_rates_m_df()# monthly policy rates, columns like EUR, GBP, ...

# forward-fill monthly to daily calendar, then align to FX columns by 3-letter prefix
rates_d = rates_m.reindex(fx.index, method="ffill")
pair2ccy = {c: c[:3] for c in fx.columns}
rate_cols = [pair2ccy[c] for c in fx.columns if pair2ccy[c] in rates_d.columns]
rates_fx = rates_d[rate_cols].copy()
rates_fx.columns = fx.columns  # align names to pairs for convenience

print("fx:", fx.shape, "| rates_m:", rates_m.shape, "| rates_fx (daily-aligned):", rates_fx.shape)
print("done")